In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report


# <h1 style='font-size:30px;'>Data Featuring</h1>

# <h2 style='font-size:25px;'>Import</h2>

In [2]:
X_train_full = pd.read_csv("../data/X_train.csv", index_col=0)
X_test_full  = pd.read_csv("../data/X_test.csv", index_col=0)

y_train = pd.read_csv("../data/y_train.csv", index_col=0).squeeze()
y_test  = pd.read_csv("../data/y_test.csv", index_col=0).squeeze()

# Make stripped-down versions for the model
non_features = ["Div", "Date", 
                "HomeTeam_ShotOnTarget", "AwayTeam_ShotOnTarget"]

X_train_features = X_train_full.drop(columns=non_features)
X_test_features  = X_test_full.drop(columns=non_features)

# Save the feature order from the stripped-down version
feature_columns = X_train_features.columns
feature_columns.to_series(name="feature").to_csv("../data/feature_columns.csv", index=False)


In [3]:
import pandas as pd

feat_order = pd.read_csv("../data/feature_columns.csv")["feature"].tolist()

def align_features(df: pd.DataFrame, feat_order: list[str]) -> pd.DataFrame:
    # add any missing training columns as zeros
    missing = [c for c in feat_order if c not in df.columns]
    if missing:
        df = df.copy()
        for c in missing:
            df[c] = 0.0

    # drop any extras not used in training
    extra = [c for c in df.columns if c not in feat_order]
    if extra:
        df = df.drop(columns=extra, errors="ignore")

    # put in the exact training order
    df = df.reindex(columns=feat_order, fill_value=0.0)
    return df

# build the actual matrices the model will see
X_train = align_features(X_train_features, feat_order)
X_test  = align_features(X_test_features,  feat_order)

# optional safety check
assert list(X_train.columns) == feat_order == list(X_test.columns)



# <h2 style='font-size:25px;'>Probabilisitc Data Selection</h2>

In [4]:
def brier_score_multi(p, y):
    Y = pd.get_dummies(y).reindex(columns=[0,1,2], fill_value=0).values
    return float(np.mean(np.sum((p - Y)**2, axis=1)))

def group_brier_scores(X, y):
    rows = []
    # consensus: probabilities already
    if set(['pH_mean','pD_mean','pA_mean']).issubset(X.columns):
        P = X[['pA_mean','pD_mean','pH_mean']].values  # order: 0,1,2
        P = np.clip(P, 1e-6, 1-1e-6)
        P = P / P.sum(axis=1, keepdims=True)
        rows.append({'group':'consensus', 'brier': brier_score_multi(P, y)})
    # raw odds approximate probabilities via 1/odds then renormalize
    triplets = [('B365A','B365D','B365H'), ('PSA','PSD','PSH'), ('MaxA','MaxD','MaxH'), ("BWH","BWD","BWA"), ( "IWH","IWD","IWA"), 
                ("WHH","WHD","WHA"), ("VCH","VCD","VCA"), ("PSCH","PSCD","PSCA")]
    for nameA,nameD,nameH in triplets:
        if {nameA,nameD,nameH}.issubset(X.columns):
            P = 1.0 / X[[nameA,nameD,nameH]].values
            P = np.clip(P, 1e-6, None)
            P = P / P.sum(axis=1, keepdims=True)
            rows.append({'group': f'odds_{nameH[:-1]}', 'brier': brier_score_multi(P, y)})
    return pd.DataFrame(rows).sort_values('brier')

brier_df = group_brier_scores(X_train, y_train)
print(brier_df)

       group     brier
0  consensus  0.557498
6    odds_WH  0.583697
8   odds_PSC  0.584901
7    odds_VC  0.585204
4    odds_BW  0.587706
5    odds_IW  0.589967
1  odds_B365  0.863573
2    odds_PS  0.865276
3   odds_Max  0.872169


Drop bad odds

In [5]:
exclude = ["B365H","B365D","B365A", "PSH","PSD","PSA", "MaxH","MaxD","MaxA"]
X_train = X_train.drop(columns=exclude)
X_test = X_test.drop(columns=exclude)

# <h2 style='font-size:25px;'>Data Selection</h2>

In [6]:
# ---- Define feature groups ----

# Form features
form_features = [
    "HomeTeam_points", "AwayTeam_points",
    "HomeTeam_avg_goal_diff", "AwayTeam_avg_goal_diff"
]

# Raw bookmaker odds (all H/D/A triplets)
bookmaker_odds = [
    "BWH","BWD","BWA",
    "IWH","IWD","IWA",
    "WHH","WHD","WHA",
    "VCH","VCD","VCA",
    "PSCH","PSCD","PSCA"
]

# Overrounds (one per bookmaker set)
overrounds = [
    "B365_overround","BW_overround","IW_overround","WH_overround",
    "VC_overround","Max_overround","PS_overround","PSC_overround"
]

# Elo ratings
elo_features = ["home_elo","away_elo","elo_diff"]

# Consensus features
consensus_features = ["pH_mean","pD_mean","pA_mean","overround_mean","overround_std"]

# Engineered extras
engineered_features = ["home_adv","draw_tightness"]

# <h3 style='font-size:20px;'>Informative groups</h3>

In [7]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pandas as pd

def group_mutual_info(X, y, groups):
    scores = []
    for gname, feats in groups.items():
        feats = [f for f in feats if f in X.columns]
        if not feats:
            continue
        mi = mutual_info_classif(X[feats], y, random_state=42, discrete_features=False)
        scores.append({
            'group': gname,
            'features': len(feats),
            'mi_mean': float(np.mean(mi)),
            'mi_median': float(np.median(mi)),
            'mi_top3_sum': float(np.sum(np.sort(mi)[-3:])),
        })
    return pd.DataFrame(scores).sort_values(['mi_top3_sum','mi_mean'], ascending=False)

groups = {
    'form': form_features,
    'bookmaker_odds': bookmaker_odds,
    'overrounds': overrounds,
    'elo': elo_features,
    'consensus': consensus_features,
    'engineered': engineered_features
}

mi_df = group_mutual_info(X_train, y_train, groups)
print(mi_df)

            group  features   mi_mean  mi_median  mi_top3_sum
1  bookmaker_odds        15  0.079875   0.100196     0.346286
4       consensus         3  0.080426   0.107707     0.241277
3             elo         3  0.059961   0.048379     0.179883
5      engineered         2  0.065186   0.065186     0.130372
0            form         4  0.023051   0.024665     0.084605
2      overrounds         8  0.011276   0.011363     0.063325


# <h3 style='font-size:20px;'>Group redundancy</h3>

In [8]:
def group_redundancy(X, groups):
    rows = []
    for gname, feats in groups.items():
        feats = [f for f in feats if f in X.columns]
        if len(feats) < 2:
            rows.append({'group': gname, 'intragroup_corr_mean': 0.0})
            continue
        corr = X[feats].corr().abs()
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        mean_corr = upper.stack().mean() if upper.size else 0.0
        rows.append({'group': gname, 'intragroup_corr_mean': float(mean_corr)})
    return pd.DataFrame(rows)

redundancy_df = group_redundancy(X_train, groups)
print(redundancy_df)

            group  intragroup_corr_mean
0            form              0.309717
1  bookmaker_odds              0.659672
2      overrounds              0.528862
3             elo              0.474944
4       consensus              0.531874
5      engineered              0.330012


# <h3 style='font-size:20px;'>Temporal Stability</h3>

In [9]:
import numpy as np
import pandas as pd

def psi_with_common_bins(a, b, bins=10):
    a = pd.Series(a, dtype='float64').dropna()
    b = pd.Series(b, dtype='float64').dropna()
    if len(a) < 20 or len(b) < 20:
        return 0.0
    s = pd.concat([a, b])
    if s.nunique(dropna=True) <= 2 or s.std() == 0:
        return 0.0

    qs = np.linspace(0, 1, bins + 1)
    edges = np.unique(np.nanquantile(s, qs))
    if len(edges) < 3:
        return 0.0

    ca = pd.cut(a, edges, include_lowest=True)
    cb = pd.cut(b, edges, include_lowest=True)

    pa = ca.value_counts(normalize=True, sort=False)
    pb = cb.value_counts(normalize=True, sort=False)

    idx = pa.index.union(pb.index)
    pa = pa.reindex(idx, fill_value=0).astype('float64') + 1e-6
    pb = pb.reindex(idx, fill_value=0).astype('float64') + 1e-6

    return float(np.sum((pa - pb) * np.log(pa / pb)))

def group_stability_no_date_common_bins(X, groups, split_ratio=0.7, bins=10, verbose=False):
    n = len(X)
    cut = int(n * split_ratio)
    early, late = X.iloc[:cut], X.iloc[cut:]

    # keep only numeric columns once to avoid repeated inference
    numeric_cols = set(X.select_dtypes(include=[np.number]).columns)

    rows = []
    for gname, feats in groups.items():
        feats = [f for f in feats if f in X.columns]
        feats = [f for f in feats if f in numeric_cols]  # ensure numeric
        psis, used = [], 0

        for f in feats:
            a = pd.to_numeric(early[f], errors='coerce')
            b = pd.to_numeric(late[f], errors='coerce')
            if a.dropna().empty or b.dropna().empty:
                continue
            try:
                psis.append(psi_with_common_bins(a.values, b.values, bins=bins))
                used += 1
            except Exception:
                continue

        psi_mean = float(np.mean(psis)) if used > 0 else 0.0  # default to 0 (stable) if none usable
        if verbose:
            print(f"{gname}: features={len(feats)}, used={used}, psi_mean={psi_mean:.4f}")

        rows.append({'group': gname, 'psi_mean': psi_mean, 'features': len(feats), 'used': used})

    return pd.DataFrame(rows).sort_values('psi_mean')

# Usage
groups = {
    'form': form_features,
    'bookmaker_odds': bookmaker_odds,
    'overrounds': overrounds,
    'elo': elo_features,
    'consensus': consensus_features,
    'engineered': engineered_features
}
stability_df = group_stability_no_date_common_bins(X_train, groups, split_ratio=0.7, bins=10, verbose=True)
print(stability_df)

form: features=4, used=4, psi_mean=0.0149
bookmaker_odds: features=15, used=15, psi_mean=0.0750
overrounds: features=8, used=8, psi_mean=4.3166
elo: features=3, used=3, psi_mean=0.1937
consensus: features=3, used=3, psi_mean=0.0850
engineered: features=2, used=2, psi_mean=0.1131
            group  psi_mean  features  used
0            form  0.014941         4     4
1  bookmaker_odds  0.075031        15    15
4       consensus  0.084992         3     3
5      engineered  0.113055         2     2
3             elo  0.193702         3     3
2      overrounds  4.316615         8     8


# <h3 style='font-size:20px;'>Cross Group Correlation</h3>

In [10]:
def cross_group_corr(X, groups):
    # mean absolute correlation between every pair of groups
    names = list(groups.keys())
    data = []
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            gi = [f for f in groups[names[i]] if f in X.columns]
            gj = [f for f in groups[names[j]] if f in X.columns]
            if not gi or not gj:
                continue
            corr = X[gi+gj].corr().abs().loc[gi, gj].values
            data.append({'g1':names[i], 'g2':names[j], 'mean_abs_corr': float(np.nanmean(corr))})
    return pd.DataFrame(data).sort_values('mean_abs_corr')

overlap_df = cross_group_corr(X_train, groups)
print(overlap_df.head(10))

                g1              g2  mean_abs_corr
1             form      overrounds       0.023592
5   bookmaker_odds      overrounds       0.039066
10      overrounds       consensus       0.039583
11      overrounds      engineered       0.046445
9       overrounds             elo       0.104737
4             form      engineered       0.376694
2             form             elo       0.387839
0             form  bookmaker_odds       0.400544
3             form       consensus       0.402658
13             elo      engineered       0.575139


# <h3 style='font-size:20px;'>Rank Group</h3>

In [11]:
import pandas as pd
import numpy as np

def rank_groups_multi(mi_df, redundancy_df, overlap_df, stability_df,
                      mi_col="mi_top3_sum",
                      weights=None,          # e.g. {"mi":2, "redundancy":1, "overlap":1, "stability":1}
                      sort_by="overall"):    # "overall", "mi", "redundancy", "overlap", or "stability"
    """
    Combine group diagnostics and produce ranks per criterion (and optional overall rank).

    Parameters
    ----------
    mi_df : DataFrame with columns ["group", mi_col] where mi_col is e.g. "mi_top3_sum"
    redundancy_df : DataFrame with columns ["group", "intragroup_corr_mean"]
    overlap_df : DataFrame with columns ["g1","g2","mean_abs_corr"] (pairwise group overlaps)
    stability_df : DataFrame with columns ["group","psi_mean"]
    mi_col : str, column name in mi_df to use for MI ranking
    weights : dict or None, weights for overall rank (keys: "mi","redundancy","overlap","stability")
    sort_by : str, which rank to sort by ("overall" requires weights)

    Returns
    -------
    DataFrame with raw metrics, ranks per criterion, and optional overall rank.
    """

    # --- Cross-group overlap → per-group mean overlap (symmetrize pairs) ---
    if overlap_df is not None and not overlap_df.empty:
        og = pd.concat([
            overlap_df.rename(columns={"g1": "group", "g2": "other"}),
            overlap_df.rename(columns={"g2": "group", "g1": "other"})
        ], ignore_index=True)
        cross_mean = (og.groupby("group")["mean_abs_corr"]
                        .mean()
                        .rename("crossgroup_corr_mean")
                        .reset_index())
    else:
        cross_mean = pd.DataFrame(columns=["group", "crossgroup_corr_mean"])

    # --- Merge everything on 'group' ---
    cols_mi = ["group", mi_col]
    cols_rd = ["group", "intragroup_corr_mean"]
    cols_st = ["group", "psi_mean"]

    df = (mi_df[cols_mi]
            .merge(redundancy_df[cols_rd], on="group", how="outer")
            .merge(cross_mean, on="group", how="outer")
            .merge(stability_df[cols_st], on="group", how="outer"))

    # Helper to rank while pushing NaNs to the bottom (worst)
    def _rank(series, ascending):
        s = series.copy()
        if s.isna().any():
            fill = (s.max() + 1) if ascending else (s.min() - 1)
            s = s.fillna(fill)
        return s.rank(ascending=ascending, method="min")

    # --- Per-criterion ranks ---
    df["rank_mi"]          = _rank(df[mi_col], ascending=False)              # higher MI is better
    df["rank_redundancy"]  = _rank(df["intragroup_corr_mean"], ascending=True)
    df["rank_overlap"]     = _rank(df["crossgroup_corr_mean"], ascending=True)
    df["rank_stability"]   = _rank(df["psi_mean"], ascending=True)

    # --- Optional overall rank (weighted sum of ranks; lower = better) ---
    if weights:
        w_mi  = float(weights.get("mi", 1.0))
        w_rd  = float(weights.get("redundancy", 1.0))
        w_ov  = float(weights.get("overlap", 1.0))
        w_ps  = float(weights.get("stability", 1.0))
        df["overall_rank_score"] = (
            w_mi*df["rank_mi"] +
            w_rd*df["rank_redundancy"] +
            w_ov*df["rank_overlap"] +
            w_ps*df["rank_stability"]
        )
        df["rank_overall"] = df["overall_rank_score"].rank(ascending=True, method="min")

    # --- Sorting ---
    sort_map = {
        "overall": "rank_overall",
        "mi": "rank_mi",
        "redundancy": "rank_redundancy",
        "overlap": "rank_overlap",
        "stability": "rank_stability",
    }
    sort_col = sort_map.get(sort_by, "rank_mi")
    if sort_by == "overall" and not weights:
        sort_col = "rank_mi"  # fallback if weights not provided

    # Nice column order
    out_cols = ["group", mi_col, "intragroup_corr_mean", "crossgroup_corr_mean", "psi_mean",
                "rank_mi", "rank_redundancy", "rank_overlap", "rank_stability"]
    if weights:
        out_cols += ["overall_rank_score", "rank_overall"]

    return df[out_cols].sort_values(sort_col, ascending=True).reset_index(drop=True)

In [12]:
# Choose weights (tweak as you like)
weights = {"mi": 2.0, "redundancy": 1.0, "overlap": 1.0, "stability": 1.0}

ranked = rank_groups_multi(
    mi_df=mi_df,
    redundancy_df=redundancy_df,
    overlap_df=overlap_df,
    stability_df=stability_df,   # from your PSI-with-common-bins function
    mi_col="mi_top3_sum",
    weights=weights,
    sort_by="overall"            # or "mi"/"redundancy"/"overlap"/"stability"
)
print(ranked)

            group  mi_top3_sum  intragroup_corr_mean  crossgroup_corr_mean  \
0            form     0.084605              0.309717              0.318265   
1  bookmaker_odds     0.346286              0.659672              0.480782   
2      engineered     0.130372              0.330012              0.462187   
3       consensus     0.241277              0.531874              0.482342   
4             elo     0.179883              0.474944              0.463766   
5      overrounds     0.063325              0.528862              0.050685   

   psi_mean  rank_mi  rank_redundancy  rank_overlap  rank_stability  \
0  0.014941      5.0              1.0           2.0             1.0   
1  0.075031      1.0              6.0           5.0             2.0   
2  0.113055      4.0              2.0           3.0             4.0   
3  0.084992      2.0              5.0           6.0             3.0   
4  0.193702      3.0              3.0           4.0             5.0   
5  4.316615      6.0       

In [13]:
from sklearn.preprocessing import LabelEncoder

feat_cats  = ["HomeTeam", "AwayTeam"]              
feat_nums  = form_features
feat_all   = feat_cats + feat_nums + bookmaker_odds

Xtr = X_train[feat_all].copy()
Xte = X_test[feat_all].copy()
ytr, yte = y_train, y_test

for c in feat_cats:
    Xtr[c] = Xtr[c].astype(str)
    Xte[c] = Xte[c].astype(str)


In [14]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from collections import Counter

# Define feature groups
feat_cats = ["HomeTeam", "AwayTeam"]              
feat_nums = form_features
feat_all = feat_cats + feat_nums + bookmaker_odds

# Prepare data
Xtr = X_train[feat_all].copy()
Xte = X_test[feat_all].copy()
ytr, yte = y_train, y_test

# Ensure categorical features are strings
for c in feat_cats:
    Xtr[c] = Xtr[c].astype(str)
    Xte[c] = Xte[c].astype(str)

# --- ALWAYS encode categorical features first ---
encoders = {}
Xtr_enc = Xtr.copy()
Xte_enc = Xte.copy()

for c in feat_cats:
    le = LabelEncoder()
    # Fit on training data only, transform both train and test
    Xtr_enc[c] = le.fit_transform(Xtr[c])
    Xte_enc[c] = le.transform(Xte[c])  # Use transform, not fit_transform
    encoders[c] = le

In [15]:
# Get indices of categorical columns for SMOTENC
cat_idx = [Xtr_enc.columns.get_loc(c) for c in feat_cats]

# --- Apply SMOTENC ---
smote = SMOTENC(
    categorical_features=cat_idx,
    sampling_strategy="not majority",  # upsample draws & away, keep home
    random_state=42
)
Xtr_bal, ytr_bal = smote.fit_resample(Xtr_enc, ytr)

print("Before SMOTE:", Counter(ytr))
print("After SMOTE:", Counter(ytr_bal))

# --- For CatBoost: Keep encoded features ---
# CatBoost works with encoded categorical features
Xtr_final = pd.DataFrame(Xtr_bal, columns=Xtr_enc.columns)
Xte_final = Xte_enc.copy()

Before SMOTE: Counter({2: 2013, 0: 1178, 1: 989})
After SMOTE: Counter({2: 2013, 0: 2013, 1: 2013})


In [16]:
import itertools, pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, log_loss, precision_recall_fscore_support

from sklearn.utils import shuffle
Xtr_bal, ytr_bal = shuffle(Xtr_bal, ytr_bal, random_state=42)

def combos(grid):
    keys = list(grid.keys())
    for vals in itertools.product(*(grid[k] for k in keys)):
        yield dict(zip(keys, vals))


# <h1 style='font-size:30px;'>Model</h1>

In [ ]:
# feat_cats + feat_nums + bookmaker_odds

param_grid = {
    "learning_rate": [0.05, 0.1, 0.15],
    "depth": [6, 7, 8],
    "l2_leaf_reg": [3, 5, 10],
    "random_strength": [1.0, 2.0],
    "bagging_temperature": [0.5, 1.0],
    "rsm": [0.9, 1.0],
    "grow_policy": ["SymmetricTree", "Lossguide"],  # compare both
}

print(param_grid)

rows = []

for p in combos(param_grid):
    model = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="Accuracy",
        iterations=1000,
        verbose=False,
        random_state=42,
        **p
    )

    model.fit(
        Xtr_bal, ytr_bal,
        eval_set=(Xte, yte),
        use_best_model=True,
        cat_features=feat_cats        
    )

    y_pred  = model.predict(Xte)
    y_proba = model.predict_proba(Xte)
    pr, rc, f1, _ = precision_recall_fscore_support(
        yte, y_pred, zero_division=0
    )

    rows.append({
        **p,
        "acc": accuracy_score(yte, y_pred),
        "logloss": log_loss(yte, y_proba),
        "c0_rec": rc[0], "c1_rec": rc[1], "c2_rec": rc[2],
        "c0_f1":  f1[0], "c1_f1":  f1[1], "c2_f1":  f1[2],
    })

results = pd.DataFrame(rows).sort_values(["acc","logloss"], ascending=[False, True])
display(results.head(15))

{'learning_rate': [0.05, 0.1, 0.15], 'depth': [6, 7, 8], 'l2_leaf_reg': [3, 5, 10], 'random_strength': [1.0, 2.0], 'bagging_temperature': [0.5, 1.0], 'rsm': [0.9, 1.0], 'grow_policy': ['SymmetricTree', 'Lossguide']}


,learning_rate,depth,l2_leaf_reg,random_strength,bagging_temperature,rsm,grow_policy,acc,logloss,c0_rec,c1_rec,c2_rec,c0_f1,c1_f1,c2_f1
143,0.05,8,10,2.0,1.0,1.0,Lossguide,0.594444,0.983985,0.574468,0.26,0.807229,0.568421,0.351351,0.701571
177,0.10,6,10,1.0,0.5,0.9,Lossguide,0.588889,0.978427,0.489362,0.24,0.855422,0.541176,0.315789,0.713568
283,0.10,8,10,2.0,0.5,1.0,Lossguide,0.588889,0.993622,0.553191,0.22,0.831325,0.553191,0.318841,0.700508
97,0.05,8,3,1.0,0.5,0.9,Lossguide,0.588889,0.995451,0.510638,0.32,0.795181,0.545455,0.385542,0.698413
141,0.05,8,10,2.0,1.0,0.9,Lossguide,0.588889,0.996568,0.617021,0.20,0.807229,0.563107,0.303030,0.701571
373,0.15,7,10,1.0,1.0,0.9,Lossguide,0.588889,1.002191,0.574468,0.18,0.843373,0.580645,0.268657,0.700000
415,0.15,8,5,2.0,1.0,1.0,Lossguide,0.583333,0.977195,0.425532,0.22,0.891566,0.487805,0.328358,0.701422
319,0.15,6,5,2.0,1.0,1.0,Lossguide,0.583333,0.981500,0.574468,0.18,0.831325,0.551020,0.281250,0.696970
281,0.10,8,10,2.0,0.5,0.9,Lossguide,0.583333,0.982129,0.489362,0.22,0.855422,0.534884,0.314286,0.696078
129,0.05,8,10,1.0,0.5,0.9,Lossguide,0.583333,0.983935,0.553191,0.22,0.819277,0.541667,0.333333,0.686869


In [ ]:
# feat_cats + feat_nums + bookmaker_odds

param_grid = {
    "learning_rate": [0.1, 0.15, 0.2],
    "depth": [6, 7],
    "l2_leaf_reg": [2, 3, 10],
    "random_strength": [0.5, 1.0],
    "bagging_temperature": [0.3, 0.5],
    "rsm": [0.9, 1.0],
    "grow_policy": ["Lossguide"],
    "max_leaves": [32, 64]
}

rows = []

for p in combos(param_grid):
    model = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="Accuracy",
        iterations=1000,
        verbose=False,
        random_state=42,
        **p
    )

    model.fit(
        Xtr_bal, ytr_bal,
        eval_set=(Xte, yte),
        use_best_model=True,
        cat_features=feat_cats         
    )

    y_pred  = model.predict(Xte)
    y_proba = model.predict_proba(Xte)
    pr, rc, f1, _ = precision_recall_fscore_support(
        yte, y_pred, zero_division=0
    )

    rows.append({
        **p,
        "acc": accuracy_score(yte, y_pred),
        "logloss": log_loss(yte, y_proba),
        "c0_rec": rc[0], "c1_rec": rc[1], "c2_rec": rc[2],
        "c0_f1":  f1[0], "c1_f1":  f1[1], "c2_f1":  f1[2],
    })

results = pd.DataFrame(rows).sort_values(["acc","logloss"], ascending=[False, True])
display(results.head(15))

,learning_rate,depth,l2_leaf_reg,random_strength,bagging_temperature,rsm,grow_policy,max_leaves,acc,logloss,c0_rec,c1_rec,c2_rec,c0_f1,c1_f1,c2_f1
252,0.20,7,2,1.0,0.5,0.9,Lossguide,32,0.611111,0.983603,0.723404,0.22,0.783133,0.618182,0.297297,0.738636
150,0.15,7,2,0.5,0.5,1.0,Lossguide,32,0.594444,0.984467,0.617021,0.28,0.771084,0.591837,0.358974,0.695652
28,0.10,6,3,1.0,0.5,0.9,Lossguide,32,0.588889,0.980267,0.531915,0.22,0.843373,0.561798,0.314286,0.696517
26,0.10,6,3,1.0,0.3,1.0,Lossguide,32,0.583333,0.990065,0.468085,0.24,0.855422,0.511628,0.342857,0.696078
54,0.10,7,2,0.5,0.5,1.0,Lossguide,32,0.583333,0.995174,0.510638,0.18,0.867470,0.545455,0.276923,0.695652
134,0.15,6,10,0.5,0.5,1.0,Lossguide,32,0.583333,1.004221,0.638298,0.24,0.759036,0.545455,0.328767,0.711864
170,0.15,7,3,1.0,0.3,1.0,Lossguide,32,0.583333,1.005975,0.617021,0.24,0.771084,0.563107,0.347826,0.680851
153,0.15,7,2,1.0,0.3,0.9,Lossguide,64,0.577778,0.980976,0.617021,0.16,0.807229,0.580000,0.238806,0.694301
135,0.15,6,10,0.5,0.5,1.0,Lossguide,64,0.577778,0.984662,0.489362,0.26,0.819277,0.522727,0.346667,0.690355
52,0.10,7,2,0.5,0.5,0.9,Lossguide,32,0.577778,0.984900,0.553191,0.18,0.831325,0.525253,0.268657,0.711340


In [ ]:
# feat_cats + feat_nums + bookmaker_odds

param_grid = {
    "learning_rate": [0.15, 0.16, 0.17, 0.18],
    "depth": [6, 7],
    "l2_leaf_reg": [1, 2],
    "random_strength": [1.0],
    "bagging_temperature": [0.2, 0.25, 0.3],
    "rsm": [0.9, 0.95, 1.0],
    "grow_policy": ["Lossguide"],
    "max_leaves": [16, 20, 24, 32]
}

rows = []

for p in combos(param_grid):
    model = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="Accuracy",
        iterations=1000,
        verbose=False,
        random_state=42,
        **p
    )

    model.fit(
        Xtr_bal, ytr_bal,
        eval_set=(Xte, yte),
        use_best_model=True,
        cat_features=feat_cats
    )

    y_pred  = model.predict(Xte)
    y_proba = model.predict_proba(Xte)
    pr, rc, f1, _ = precision_recall_fscore_support(
        yte, y_pred, zero_division=0
    )

    rows.append({
        **p,
        "acc": accuracy_score(yte, y_pred),
        "logloss": log_loss(yte, y_proba),
        "c0_rec": rc[0], "c1_rec": rc[1], "c2_rec": rc[2],
        "c0_f1":  f1[0], "c1_f1":  f1[1], "c2_f1":  f1[2],
    })

results = pd.DataFrame(rows).sort_values(["acc","logloss"], ascending=[False, True])
display(results.head(15))

,learning_rate,depth,l2_leaf_reg,random_strength,bagging_temperature,rsm,grow_policy,max_leaves,acc,logloss,c0_rec,c1_rec,c2_rec,c0_f1,c1_f1,c2_f1
223,0.16,7,1,1.0,0.20,0.95,Lossguide,32,0.611111,0.959579,0.617021,0.38,0.746988,0.574257,0.452381,0.708571
13,0.15,6,1,1.0,0.25,0.90,Lossguide,20,0.611111,0.982685,0.638298,0.26,0.807229,0.582524,0.376812,0.712766
284,0.16,7,2,1.0,0.30,1.00,Lossguide,16,0.605556,0.972337,0.680851,0.24,0.783133,0.621359,0.342857,0.695187
537,0.18,7,1,1.0,0.30,1.00,Lossguide,20,0.605556,0.995310,0.595745,0.22,0.843373,0.577320,0.314286,0.725389
407,0.17,7,2,1.0,0.20,1.00,Lossguide,32,0.600000,0.965037,0.638298,0.30,0.759036,0.588235,0.405405,0.684783
32,0.15,6,1,1.0,0.30,1.00,Lossguide,16,0.600000,0.974729,0.510638,0.30,0.831325,0.539326,0.405405,0.700508
67,0.15,6,2,1.0,0.30,0.95,Lossguide,32,0.600000,0.981227,0.553191,0.26,0.831325,0.571429,0.366197,0.696970
148,0.16,6,1,1.0,0.20,0.95,Lossguide,16,0.600000,0.981863,0.595745,0.22,0.831325,0.560000,0.333333,0.711340
256,0.16,7,2,1.0,0.20,0.95,Lossguide,16,0.600000,0.990963,0.617021,0.22,0.819277,0.557692,0.333333,0.715789
332,0.17,6,2,1.0,0.20,1.00,Lossguide,16,0.594444,0.974648,0.531915,0.24,0.843373,0.537634,0.342857,0.710660


In [ ]:
# feat_cats + feat_nums + bookmaker_odds

param_grid = {
    "grow_policy": ["Lossguide"],
    "depth": [6, 7],
    "learning_rate": [0.16, 0.17, 0.18],            
    "l2_leaf_reg": [2.0, 2.5, 3],
    "min_data_in_leaf": [5, 8, 12],
    "max_leaves": [20, 24, 28],
    "rsm": [0.85, 0.87, 0.90],
    "bagging_temperature": [0.20, 0.25, 0.3],
    "random_strength": [0.5, 1.0],
}



rows = []

for p in combos(param_grid):
    model = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="Accuracy",
        iterations=1000,
        verbose=False,
        random_state=42,
        **p
    )

    model.fit(
        Xtr_bal, ytr_bal,
        eval_set=(Xte, yte),
        use_best_model=True,
        cat_features=feat_cats
    )

    y_pred  = model.predict(Xte)
    y_proba = model.predict_proba(Xte)
    pr, rc, f1, _ = precision_recall_fscore_support(
        yte, y_pred, zero_division=0
    )

    rows.append({
        **p,
        "acc": accuracy_score(yte, y_pred),
        "logloss": log_loss(yte, y_proba),
        "c0_rec": rc[0], "c1_rec": rc[1], "c2_rec": rc[2],
        "c0_f1":  f1[0], "c1_f1":  f1[1], "c2_f1":  f1[2],
    })

results = pd.DataFrame(rows).sort_values(["acc","logloss"], ascending=[False, True])
display(results.head(15))

,grow_policy,depth,learning_rate,l2_leaf_reg,min_data_in_leaf,max_leaves,rsm,bagging_temperature,random_strength,acc,logloss,c0_rec,c1_rec,c2_rec,c0_f1,c1_f1,c2_f1
1729,Lossguide,7,0.16,2.5,12,20,0.85,0.20,1.0,0.622222,0.971068,0.680851,0.32,0.771084,0.609524,0.410256,0.723164
1297,Lossguide,6,0.18,3.0,5,20,0.85,0.20,1.0,0.611111,0.962962,0.638298,0.24,0.819277,0.594059,0.347826,0.715789
1748,Lossguide,7,0.16,2.5,12,24,0.85,0.25,0.5,0.611111,0.976561,0.595745,0.26,0.831325,0.571429,0.388060,0.707692
2386,Lossguide,7,0.17,3.0,12,20,0.87,0.30,0.5,0.611111,0.981151,0.595745,0.36,0.771084,0.583333,0.444444,0.699454
1900,Lossguide,7,0.16,3.0,12,20,0.87,0.30,0.5,0.611111,0.989073,0.659574,0.36,0.734940,0.558559,0.433735,0.734940
1356,Lossguide,6,0.18,3.0,8,20,0.87,0.20,0.5,0.605556,0.973894,0.659574,0.30,0.759036,0.620000,0.379747,0.696133
2482,Lossguide,7,0.18,2.0,5,28,0.90,0.30,0.5,0.605556,0.978975,0.723404,0.32,0.710843,0.612613,0.421053,0.682081
2675,Lossguide,7,0.18,2.5,8,24,0.87,0.30,1.0,0.605556,0.983217,0.510638,0.30,0.843373,0.551724,0.405405,0.703518
2238,Lossguide,7,0.17,2.5,12,24,0.87,0.20,0.5,0.605556,0.990319,0.574468,0.28,0.819277,0.562500,0.400000,0.701031
1920,Lossguide,7,0.16,3.0,12,24,0.90,0.20,0.5,0.605556,1.002430,0.680851,0.18,0.819277,0.621359,0.281250,0.704663


In [21]:
# feat_cats + feat_nums + bookmaker_odds

from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=1000, loss_function="MultiClass", grow_policy='Lossguide', random_strength=1.0, bagging_temperature=0.20, rsm=0.85,
                           eval_metric="Accuracy", learning_rate=0.16, random_state=42, depth=7,
                           l2_leaf_reg=2.5, min_data_in_leaf=12, max_leaves=20)

model.fit(Xtr_bal,
          ytr_bal,
          eval_set=(Xte, yte),
          cat_features=feat_cats,    
          verbose=False)

y_pred = model.predict(Xte)
print(f"Test Set Accuracy: {accuracy_score(yte, y_pred):.2f}")
print(classification_report(yte, y_pred, zero_division=0))
print(model.get_best_score())

Test Set Accuracy: 0.62
              precision    recall  f1-score   support

           0       0.55      0.68      0.61        47
           1       0.57      0.32      0.41        50
           2       0.68      0.77      0.72        83

    accuracy                           0.62       180
   macro avg       0.60      0.59      0.58       180
weighted avg       0.62      0.62      0.61       180

{'learn': {'Accuracy': 0.98642159297897, 'MultiClass': 0.24309875922298613}, 'validation': {'Accuracy': 0.6222222222222222, 'MultiClass': 0.9697007690746107}}


# Model

In [221]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score

def run_model(classifier, param_grid, X_train, y_train, X_test, y_test):

    grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='balanced_accuracy',
                               cv=5, verbose=1)
    grid_search.fit(X_train, y_train)
    model = grid_search.best_estimator_
    y_pred = model.predict(X_test)
    print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(classification_report(y_test, y_pred, zero_division=0))

    return grid_search.best_params_

# <h1 style='font-size:30px;'>Random Forest Classifier</h1>

In [226]:
# New

from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators': [100, 500, 1000],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    }
classifier = RandomForestClassifier(random_state=42)
best_params_rf = run_model(classifier, param_grid, Xtr_enc, ytr, Xte_enc, yte)
best_params_rf

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Test Set Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.53      0.43      0.47        47
           1       0.48      0.26      0.34        50
           2       0.57      0.80      0.67        83

    accuracy                           0.55       180
   macro avg       0.53      0.49      0.49       180
weighted avg       0.54      0.55      0.52       180



{'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}

In [229]:
classifier = RandomForestClassifier(random_state=42, n_estimators=100, min_samples_leaf=4, min_samples_split=10)
classifier.fit(Xtr_enc, ytr)
y_pred = classifier.predict(Xte_enc)
print(f"Test Set Accuracy: {accuracy_score(yte, y_pred):.2f}")
print(classification_report(yte, y_pred, zero_division=0))

Test Set Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.53      0.43      0.47        47
           1       0.48      0.26      0.34        50
           2       0.57      0.80      0.67        83

    accuracy                           0.55       180
   macro avg       0.53      0.49      0.49       180
weighted avg       0.54      0.55      0.52       180



# <h1 style='font-size:30px;'>Gradient Boosting Classifier</h1>

In [231]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=1000, max_depth=10)
model.fit(Xtr_enc, ytr)
y_pred = model.predict(Xte_enc)
print(f"Test Set Accuracy: {accuracy_score(yte, y_pred):.2f}")
print(classification_report(yte, y_pred, zero_division=0))

Test Set Accuracy: 0.51
              precision    recall  f1-score   support

           0       0.47      0.43      0.44        47
           1       0.39      0.22      0.28        50
           2       0.55      0.72      0.62        83

    accuracy                           0.51       180
   macro avg       0.47      0.46      0.45       180
weighted avg       0.48      0.51      0.48       180



# <h1 style='font-size:30px;'>Naive Bayes</h1>

In [ ]:
from sklearn.naive_bayes import GaussianNB

param_grid = {
    'var_smoothing': np.logspace(0, -12, num=10),
    'priors': [[0.3, 0.4, 0.3]]
}
classifier = GaussianNB()

best_params_nb = run_model(classifier, param_grid, Xtr_enc, ytr, Xte_enc, yte)
best_params_nb

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Test Set Accuracy: 0.49
              precision    recall  f1-score   support

           0       0.46      0.55      0.50        47
           1       0.34      0.46      0.39        50
           2       0.70      0.47      0.56        83

    accuracy                           0.49       180
   macro avg       0.50      0.49      0.48       180
weighted avg       0.54      0.49      0.50       180



{'priors': [0.3, 0.4, 0.3], 'var_smoothing': 2.1544346900318868e-11}

# <h1 style='font-size:30px;'>Stacking Classifier</h1>

# <h1 style='font-size:15px;'>By Using Stacking Classifier, we can have a more balanced result which have a better performance in predicting results for Draw</h1>

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

classifier_1 = GradientBoostingClassifier(n_estimators=1000, criterion='friedman_mse', learning_rate=0.1, subsample=0.5)
classifier_2 = RandomForestClassifier(n_estimators=1000, min_samples_leaf=1, max_leaf_nodes=5)
classifier_3 = GaussianNB(var_smoothing=1e-09)
sclf = StackingClassifier(estimators = [('rf', classifier_2), ('gb', classifier_1), ('gnb', classifier_3)],
                          final_estimator = classifier_3
                          )

model = sclf.fit(Xtr_enc, ytr)
y_pred = model.predict(X_test)
print(f"Test Set Accuracy: {accuracy_score(yte, y_pred):.2f}")
print(classification_report(yte, y_pred, zero_division=0))

Test Set Accuracy: 0.53
              precision    recall  f1-score   support

           0       0.46      0.62      0.53        47
           1       0.42      0.30      0.35        50
           2       0.64      0.63      0.63        83

    accuracy                           0.53       180
   macro avg       0.51      0.51      0.50       180
weighted avg       0.53      0.53      0.53       180



# <h1 style='font-size:30px;'>CatBoost</h1>

# <h1 style='font-size:30px;'>XGBoost</h1>

In [37]:
from xgboost import XGBClassifier

grid_params = {
    'max_depth': [3,6,9],
    'min_child_weight': [1,3,5],
    'learning_rate': [0.1, 0.5, 1],
    'objective': ['multi:softmax'],
    'n_estimators': [1000]
}

classifier = XGBClassifier()
best_params_gb = run_model(classifier, grid_params, X_train, y_train, X_test, y_test)
best_params_gb

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Test Set Accuracy: 0.52
              precision    recall  f1-score   support

           0       0.48      0.43      0.45        47
           1       0.37      0.20      0.26        50
           2       0.57      0.76      0.65        83

    accuracy                           0.52       180
   macro avg       0.47      0.46      0.45       180
weighted avg       0.49      0.52      0.49       180



{'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 5,
 'n_estimators': 1000,
 'objective': 'multi:softmax'}

# <h1 style='font-size:30px;'>Save Model</h1>

# <h1 style='font-size:15px;'>Catboost is the most efficient model such that it has best balanced prediction result in all 3 possible outcomes</h1>

In [19]:
import pickle

catboost = CatBoostClassifier(iterations=1000, loss_function="MultiClass", 
                                eval_metric="Accuracy", learning_rate=0.3, l2_leaf_reg=9, class_weights=[1, 1.5, 1])

catboost.fit(X_train,
          y_train,
          eval_set=(X_test, y_test),
          verbose=False)

pickle.dump(catboost, open('catboost.pkl', 'wb'))